In [3]:
f = open('ratings.dat', 'r')
data = f.readlines()
f = open('movies.dat', 'r', encoding='ISO-8859-1')
movies = f.readlines()
movie_names = [_.split('::')[1] for _ in movies]
movie_ids = [_.split('::')[0] for _ in movies]
movie_dict = dict(zip(movie_ids, movie_names))
id_mapping = dict(zip(movie_ids, range(len(movie_ids))))

In [ ]:

interaction_dicts = dict()
for line in data:
    user_id, movie_id, rating, timestamp = line.split('::')
    if user_id not in interaction_dicts:
        interaction_dicts[user_id] = {
            'movie_id': [],
            'rating': [],
            'timestamp': [],
            'movie_title': [],
        }
    interaction_dicts[user_id]['movie_id'].append(movie_id)
    interaction_dicts[user_id]['rating'].append(int(float(rating) > 3.0))
    interaction_dicts[user_id]['timestamp'].append(timestamp)
    interaction_dicts[user_id]['movie_title'].append(movie_dict[movie_id])


In [ ]:
with open('all.csv', 'w') as f:
    import csv
    writer = csv.writer(f)
    writer.writerow(['user_id', 'item_id', 'rating', 'timestamp', 'item_title'])
    for user_id, user_dict in interaction_dicts.items():
        writer.writerow([user_id, user_dict['movie_id'], user_dict['rating'], user_dict['timestamp'], user_dict['movie_title']])
    

In [ ]:
sequential_interaction_list = []
seq_len = 10
for user_id in interaction_dicts:
    temp = zip(interaction_dicts[user_id]['movie_id'], interaction_dicts[user_id]['rating'], interaction_dicts[user_id]['timestamp'])
    temp = sorted(temp, key=lambda x: x[2])
    result = zip(*temp)
    interaction_dicts[user_id]['movie_id'], interaction_dicts[user_id]['rating'], interaction_dicts[user_id]['timestamp'] = [list(_) for _ in result]
    for i in range(10, len(interaction_dicts[user_id]['movie_id'])):
        if interaction_dicts[user_id]['rating'][i] == 0:
            continue
        sequential_interaction_list.append(
            [user_id, interaction_dicts[user_id]['movie_id'][i-seq_len:i], interaction_dicts[user_id]['rating'][i-seq_len:i], interaction_dicts[user_id]['movie_id'][i], interaction_dicts[user_id]['rating'][i], interaction_dicts[user_id]['timestamp'][i].strip('\n')]
        )
print(len(sequential_interaction_list))

In [ ]:
import csv
sequential_interaction_list = sorted(sequential_interaction_list, key=lambda x: int(x[-1]))
with open('./train.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['user_id', 'history_movie_id', 'history_rating', 'movie_id', 'rating', 'timestamp'])
    writer.writerows(sequential_interaction_list[:int(len(sequential_interaction_list)*0.8)])
with open('./valid.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['user_id', 'history_movie_id', 'history_rating', 'movie_id', 'rating', 'timestamp'])
    writer.writerows(sequential_interaction_list[int(len(sequential_interaction_list)*0.8):int(len(sequential_interaction_list)*0.9)])
with open('./test.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['user_id', 'history_movie_id', 'history_rating', 'movie_id', 'rating', 'timestamp'])
    writer.writerows(sequential_interaction_list[int(len(sequential_interaction_list)*0.9):])

In [4]:
import json
import pandas as pd
import random
import numpy as np
import jsonlines
# 'user_id', 'history_movie_id', 'history_rating', 'movie_id', 'rating', 'timestamp'
def user_specific_description_preprocess(input_path, output_path, sample=False):
    data = pd.read_csv(input_path)
    with jsonlines.open(output_path, 'w') as writer:
        for index, row in data.iterrows():
            row['history_movie_id'] = eval(row['history_movie_id'])
            row['history_rating'] = eval(row['history_rating'])
            L = len(row['history_movie_id'])
            preference = []
            unpreference = []
            for i in range(L):
                if int(row['history_rating'][i]) == 1:
                    preference.append(movie_dict[row['history_movie_id'][i]])
                else:
                    unpreference.append(movie_dict[row['history_movie_id'][i]])
            target_movie = movie_dict[str(row['movie_id'])]
            preference_str = ""
            unpreference_str = ""
            for i in range(len(preference)):
                if i == 0:
                    preference_str += "\"" + preference[i] + "\""
                else:
                    preference_str += ", \"" + preference[i] + "\""
            for i in range(len(unpreference)):
                if i == 0:
                    unpreference_str += "\"" + unpreference[i] + "\""
                else:
                    unpreference_str += ", \"" + unpreference[i] + "\""
            target_movie_str = "\"" + target_movie + "\""
            # json_list.append({
            #     "instruction": "Given ten movies that the user likes and dislikes, please recommend a new movie that the user likes to the user.",
            #     "input": f"User Likes: {preference_str}\nUser Dislikes: {unpreference_str}\n ",
            #     "output": target_movie_str,
            # })
            temp_inputs = f'''The user enjoy the following movies in the past: {preference_str}. 
The user doesn't like the following movies in the past: {unpreference_str}.
Now please recommend the movie the movie -- {target_movie_str} by writting a description with at most fifty words and at least thirty words. 
Try to extract similarities from the movies that users like to attract users, while avoiding content that is similar to movies that users don't like.
Please begin with \"I recommend the movie {target_movie_str} to you.\"'''
            
            writer.write({"inputs": temp_inputs})
user_specific_description_preprocess('./generative/train_balance.csv', './user_specific_pre.jsonl')

In [1]:
import json
import pandas as pd
import random
import numpy as np
# 'user_id', 'history_movie_id', 'history_rating', 'movie_id', 'rating', 'timestamp'
f = open('movie_description.txt', 'r')
movie_description = f.readlines()
f.close()
def csv_to_json(input_path, output_path, sample=False, balance=False):
    data = pd.read_csv(input_path)
    if sample and balance:
        return None
    if balance:
        item_set = set()
        data = data.sample(frac=1, random_state=42).reset_index(drop=True)
        index_list = []
        for index, row in data.iterrows():
            item_id = row['item_title']
            if item_id not in item_set:
                item_set.add(item_id)
                index_list.append(index)
        data = data.iloc[index_list]
        data.to_csv(output_path[:-5] + ".csv", index=False)
    if sample:
        data = data.sample(n=5000, random_state=42).reset_index(drop=True)
        data.to_csv(output_path[:-5] + ".csv", index=False)
    json_list = []
    for index, row in data.iterrows():
        # print(row['user_id'], row['history_movie_id'], row['history_rating'], row[
        # 'movie_id'], row['rating'], row['timestamp'])
        row['history_movie_id'] = eval(row['history_movie_id'])
        row['history_rating'] = eval(row['history_rating'])
        L = len(row['history_movie_id'])
        history = "The user has watched the following movies before:"
        for i in range(L):
            if i == 0:
                history += "\"" + row['history_movie_title'][i] + "\""
            else:
                history += ", \"" + row['history_movie_title'][i] + "\""
        target_movie_name = "\"" + movie_dict[str(row['movie_id'])] + "\""
        json_list.append({
            "instruction": "Given a list of movies the user has watched before, please recommend a new movie that the user likes to the user.",
            "input": f"{history}\n ",
            "output": target_movie_name,
        })    
        
    with open(output_path, 'w') as f:
        json.dump(json_list, f, indent=4)


In [ ]:
csv_to_json('./train.csv', './train.json')
csv_to_json('./valid.csv', './valid.json')
csv_to_json('./test.csv', './test.json')
csv_to_json('./valid.csv', './valid_balance.json', balance=True)
csv_to_json('./test.csv', './test_balance.json', balance=True)
csv_to_json('./valid.csv', './valid_5000.json', sample=True)
csv_to_json('./test.csv', './test_5000.json', sample=True)